In [ ]:
!pip3 install google-cloud-speech

     |████████████████████████████████| 92kB 3.4MB/s 


2. **Download audio files for testing**

Following files will be used as test cases for all speech recognition alternatives covered in this notebook.

In [ ]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/audio-0.6.0.tar.gz
!tar -xvzf audio-0.6.0.tar.gz
!ls -l ./audio/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   608    0   608    0     0   2632      0 --:--:-- --:--:-- --:--:--  2620
100  192k  100  192k    0     0   310k      0 --:--:-- --:--:-- --:--:--  310k
audio/
audio/2830-3980-0043.wav
audio/Attribution.txt
audio/4507-16021-0012.wav
audio/8455-210777-0068.wav
audio/License.txt
total 260
-rw-r--r-- 1 501 staff 63244 Nov 18  2017 2830-3980-0043.wav
-rw-r--r-- 1 501 staff 87564 Nov 18  2017 4507-16021-0012.wav
-rw-r--r-- 1 501 staff 82924 Nov 18  2017 8455-210777-0068.wav
-rw-r--r-- 1 501 staff   340 May 14  2018 Attribution.txt
-rw-r--r-- 1 501 staff 18652 May 12  2018 License.txt


3. **Define test cases**

In [ ]:
TESTCASES = [
  {
    'filename': 'audio/2830-3980-0043.wav',
    'text': 'experience proves this',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  },
  {
    'filename': 'audio/4507-16021-0012.wav',
    'text': 'why should one halt on the way',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  },
  {
    'filename': 'audio/8455-210777-0068.wav',
    'text': 'your power is sufficient i said',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  }
]

4. **Utility Functions**

In [ ]:
from typing import Tuple
import wave

def read_wav_file(filename) -> Tuple[bytes, int]:
    with wave.open(filename, 'rb') as w:
        rate = w.getframerate()
        frames = w.getnframes()
        buffer = w.readframes(frames)

    return buffer, rate

def simulate_stream(buffer: bytes, batch_size: int = 4096):
    buffer_len = len(buffer)
    offset = 0
    while offset < buffer_len:
        end_offset = offset + batch_size
        buf = buffer[offset:end_offset]
        yield buf
        offset = end_offset


---

# Google

Google has [speech-to-text](https://cloud.google.com/speech-to-text/docs) as one of the Google Cloud services. It has [libraries](https://cloud.google.com/speech-to-text/docs/reference/libraries) in C#, Go, Java, JavaScript, PHP, Python, and Ruby. It supports both batch and stream modes.

## Setup

1. **Upload Google Cloud Cred file**

Have Google Cloud creds stored in a file named **`gc-creds.json`**, and upload it by running following code cell. See https://developers.google.com/accounts/docs/application-default-credentials for more details.

This may reqire enabling **third-party cookies**. Check out https://colab.research.google.com/notebooks/io.ipynb for other alternatives.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving gc-creds.json to gc-creds.json
User uploaded file "gc-creds.json" with length 2314 bytes


In [ ]:
!pwd
!ls -l ./gc-creds.json

/content
-rw-r--r-- 1 root root 2314 Jan 30 00:20 ./gc-creds.json


2. **Set environment variable**

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/gc-creds.json'

!ls -l $GOOGLE_APPLICATION_CREDENTIALS

-rw-r--r-- 1 root root 2314 Jan 30 00:20 /content/gc-creds.json


## Batch API

In [ ]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums

def google_batch_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    client = speech_v1.SpeechClient()

    config = {
        'language_code': lang,
        'sample_rate_hertz': rate,
        'encoding': enums.RecognitionConfig.AudioEncoding[encoding]
    }

    audio = {
        'content': buffer
    }

    response = client.recognize(config, audio)
    # For bigger audio file, the previous line can be replaced with following:
    # operation = client.long_running_recognize(config, audio)
    # response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        return alternative.transcript

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('google-cloud-batch-stt: "{}"'.format(
        google_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
google-cloud-batch-stt: "experience proves this"

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
google-cloud-batch-stt: "why should one halt on the way"

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
google-cloud-batch-stt: "your power is sufficient I said"


## Streaming API

In [ ]:
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

def response_stream_processor(responses):
    print('interim results: ')

    transcript = ''
    num_chars_printed = 0
    for response in responses:
        if not response.results:
            continue

        result = response.results[0]
        if not result.alternatives:
            continue

        transcript = result.alternatives[0].transcript
        print('{0}final: {1}'.format(
            '' if result.is_final else 'not ',
            transcript
        ))

    return transcript

def google_streaming_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)

    client = speech.SpeechClient()

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding[encoding],
        sample_rate_hertz=rate,
        language_code=lang
    )

    streaming_config = types.StreamingRecognitionConfig(
        config=config,
        interim_results=True
    )

    audio_generator = simulate_stream(buffer)  # buffer chunk generator
    requests = (types.StreamingRecognizeRequest(audio_content=chunk) for chunk in audio_generator)
    responses = client.streaming_recognize(streaming_config, requests)
    # Now, put the transcription responses to use.
    return response_stream_processor(responses)

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('google-cloud-streaming-stt: "{}"'.format(
        google_streaming_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
interim results: 
not final: next
not final: iSpy
not final: Aspira
not final: Xperia
not final: Experian
not final: experience
not final: experience proved
not final: experience proves
not final: experience proves the
not final: experience proves that
not final: experience
final: experience proves this
google-cloud-streaming-stt: "experience proves this"

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
interim results: 
not final: what
not final: watch
not final: why should
not final: why should we
not final: why should one
not final: why should one who
not final: why should one have
not final: why should
not final: why should
not final: why should
not final: why should
not final: why should one
not final: why should one
not final: why should one
not final: why should one
not final: why should one halt
not final: why should one halt on
not final: why should one hal